In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import numpy as np
from scipy.stats import chi2_contingency

In [4]:
clean_df = pd.read_csv("C:/Users/Janak ARORA/Desktop/Ironhack_files/Final_project/tripadvisor_restaurants_analysis/data/cleaned\clean_data.csv")
clean_df

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_10984\2397448817.py:1: SyntaxWarning: invalid escape sequence '\c'
  clean_df = pd.read_csv("C:/Users/Janak ARORA/Desktop/Ironhack_files/Final_project/tripadvisor_restaurants_analysis/data/cleaned\clean_data.csv")


,restaurant_name,country,city,latitude,longitude,claimed,awards,top_tags,price_range,meals,...,total_reviews_count,default_language,excellent,terrible,primary_cuisine,cuisine_list,is_Indian,meals_list,is_english,is_vegetarian
0,Le 147,France,Saint-Jouvent,45.961674,1.169131,Claimed,Unknown,"Cheap Eats, French",Not Available,"Lunch, Dinner",...,36.0,english,2.0,0.0,French,['French'],False,"['Lunch', 'Dinner']",True,False
1,Le Saint Jouvent,France,Saint-Jouvent,45.957040,1.205480,Unclaimed,Unknown,Cheap Eats,Not Available,Unknown,...,5.0,all languages,2.0,0.0,Unknown,['Unknown'],False,[],False,False
2,Au Bout du Pont,France,Rivarennes,46.635895,1.386133,Claimed,Unknown,"Cheap Eats, French, European",Not Available,"Dinner, Lunch, Drinks",...,13.0,english,3.0,0.0,French,"['French', 'European']",False,"['Lunch', 'Dinner', 'Drinks']",True,False
3,Le Relais de Naiade,France,Lacelle,45.642610,1.824460,Claimed,Unknown,"Cheap Eats, French",Not Available,"Lunch, Dinner",...,34.0,english,1.0,0.0,French,['French'],False,"['Lunch', 'Dinner']",True,False
4,Relais Du MontSeigne,France,Saint-Laurent-de-Levezou,44.208860,2.960470,Unclaimed,Unknown,"Mid-range, French",Not Available,"Lunch, Dinner",...,11.0,all languages,4.0,0.0,French,['French'],False,"['Lunch', 'Dinner']",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326947,Domino's Pizza,United Kingdom,Goole,53.704323,-0.873573,Unclaimed,Unknown,Unknown,Not Available,"Lunch, Dinner",...,0.0,unknown,0.0,0.0,Unknown,['Unknown'],False,"['Lunch', 'Dinner']",False,False
326948,Subway,United Kingdom,Goole,53.706192,-0.914828,Unclaimed,Unknown,Unknown,Not Available,"Lunch, Dinner",...,0.0,unknown,0.0,0.0,Unknown,['Unknown'],False,"['Lunch', 'Dinner']",False,False
326949,The Kings Arms,United Kingdom,Goole,53.730110,-1.156390,Claimed,Unknown,"Mid-range, European",€12-€23,"Lunch, Dinner, Drinks",...,2.0,english,2.0,0.0,European,['European'],False,"['Lunch', 'Dinner', 'Drinks']",True,False
326950,Java Bar,United Kingdom,Rawcliffe,53.979576,-1.107181,Claimed,Unknown,Cafe,Not Available,"Breakfast, Lunch",...,5.0,english,4.0,0.0,Cafe,['Cafe'],False,"['Lunch', 'Breakfast']",True,False


In [92]:
clean_df["excellent"].unique()

array([2.000e+00, 3.000e+00, 1.000e+00, ..., 2.132e+03, 1.130e+03,
       9.650e+02], shape=(1519,))

## Create CSV for tables that does not have Foreign Keys

In [ ]:
"""
1. City: city_id, name
2. meals: meal_id, meal_type
3. review_summary: review_id, avg_rating, total_reviews_count, excellent, terrible
4. dietary_options: diet_id, vegetarian_friendly, vegan_options, gluten_free
5. price_range: price_id, price_range
6. language: language_id, name
"""

#### CITY TABLE

In [56]:
city = pd.DataFrame({"city_id": [i+1 for i in range(clean_df["city"].nunique())], "name": clean_df["city"].unique()})
city

,city_id,name
0,1,Saint-Jouvent
1,2,Rivarennes
2,3,Lacelle
3,4,Saint-Laurent-de-Levezou
4,5,Le Crozet
...,...,...
22674,22675,Orleton
22675,22676,Ashford Bowdler
22676,22677,Bromfield
22677,22678,Willaston on the Wirral


In [48]:
city.to_csv("city.csv", index=False)

#### MEALS TABLE

In [57]:
meals = pd.DataFrame({"meal_id": [i+1 for i in range(clean_df["meals"].nunique())], "meal_type": clean_df["meals"].unique()})
meals

,meal_id,meal_type
0,1,"Lunch, Dinner"
1,2,Unknown
2,3,"Dinner, Lunch, Drinks"
3,4,"Lunch, Dinner, Drinks"
4,5,"After-hours, Drinks, Lunch, Dinner, Brunch"
...,...,...
501,502,"Lunch, Drinks, Brunch, Breakfast, Dinner"
502,503,"Brunch, Lunch, Breakfast, Drinks"
503,504,"Lunch, Drinks, Breakfast, Dinner, Brunch, Afte..."
504,505,"Drinks, Breakfast, Dinner, Lunch, Brunch"


In [51]:
meals.to_csv("meals.csv", index=False)

#### REVIEW SUMMARY TABLE

In [97]:
review_summary = (
    clean_df[[
        "avg_rating",
        "total_reviews_count",
        "excellent",
        "terrible"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

review_summary.insert(0, "review_id", review_summary.index + 1)
review_summary

,review_id,avg_rating,total_reviews_count,excellent,terrible
0,1,4.0,36.0,2.0,0.0
1,2,4.0,5.0,2.0,0.0
2,3,5.0,13.0,3.0,0.0
3,4,4.0,34.0,1.0,0.0
4,5,4.5,11.0,4.0,0.0
...,...,...,...,...,...
121213,121214,4.5,252.0,179.0,3.0
121214,121215,4.5,131.0,86.0,4.0
121215,121216,4.5,265.0,170.0,5.0
121216,121217,4.0,518.0,189.0,21.0


In [96]:

"""
review_summary = pd.DataFrame({"review_id": [i+1 for i in range(clean_df["total_reviews_count"].nunique())], 
                               "avg_rating": clean_df["avg_rating"].unique(), 
                               "total_reviews_count": clean_df["total_reviews_count"].unique(), 
                               "excellent": clean_df["excellent"].unique(), 
                               "terrible": clean_df["terrible"].unique()})
review_summary
"""

'\nreview_summary = pd.DataFrame({"review_id": [i+1 for i in range(clean_df["total_reviews_count"].nunique())], \n                               "avg_rating": clean_df["avg_rating"].unique(), \n                               "total_reviews_count": clean_df["total_reviews_count"].unique(), \n                               "excellent": clean_df["excellent"].unique(), \n                               "terrible": clean_df["terrible"].unique()})\nreview_summary\n'

In [23]:
review_summary.to_csv("review_summary.csv", index=False)

#### DIETARY OPTIONS TABLE

In [87]:
dietary_options1 = pd.DataFrame({"diet_id": [i+1 for i in range(clean_df["vegetarian_friendly"].nunique())  ], 
                                "vegetarian_friendly": clean_df["vegetarian_friendly"].unique(), 
                                "vegan_options": clean_df["vegan_options"].unique(), 
                                "gluten_free": clean_df["gluten_free"].unique()})

data = {"diet_id": 3, "vegetarian_friendly": "y", "vegan_options": "Y", "gluten_free": "N"}
data1 = {"diet_id": 4, "vegetarian_friendly": "n", "vegan_options": "N", "gluten_free": "Y"}
data2 = {"diet_id": 5, "vegetarian_friendly": "y", "vegan_options": "N", "gluten_free": "Y"}

dietary_options2 = pd.DataFrame([data, data1, data2])

dietary_options = pd.concat([dietary_options1, dietary_options2], axis=0)
dietary_options.reset_index(drop = True, inplace=True)
dietary_options

,diet_id,vegetarian_friendly,vegan_options,gluten_free
0,1,n,N,N
1,2,y,Y,Y
2,3,y,Y,N
3,4,n,N,Y
4,5,y,N,Y


In [88]:
dietary_options.to_csv("dietary_options.csv", index=False)

#### PRICE RANGE TABLE

In [85]:
price_range = pd.DataFrame({"price_id": [i+1 for i in range(clean_df["price_range"].nunique())], 
                                "price_range": clean_df["price_range"].unique()})
price_range

,price_id,price_range
0,1,Not Available
1,2,€14-€29
2,3,€8-€17
3,4,€10-€35
4,5,€12-€26
...,...,...
4183,4184,"€231-€1,097"
4184,4185,€13-€68
4185,4186,€1-€8
4186,4187,€307-€827


In [65]:
price_range.to_csv("price_range.csv", index=False)

#### LANGUAGE TABLE

In [86]:
language = pd.DataFrame({"language_id": [i+1 for i in range(clean_df["default_language"].nunique())], 
                                "name": clean_df["default_language"].unique()})
language

,language_id,name
0,1,english
1,2,all languages
2,3,unknown


In [93]:
language.to_csv("language.csv", index=False)

## Create CSV for tables that does have Foreign Keys

In [ ]:
"""
1. country: country_id, name, city_id

2. location: location_id, lat, long, city_id

3. restaurant_language: id, rest_id, lang_id

4. restaurant_meals: id, rest_id, meal_id

5. restaurant_dietary_options: id, rest_id, diet_id
"""

#### COUNTRY TABLE

In [104]:
city

,city_id,name
0,1,Saint-Jouvent
1,2,Rivarennes
2,3,Lacelle
3,4,Saint-Laurent-de-Levezou
4,5,Le Crozet
...,...,...
22674,22675,Orleton
22675,22676,Ashford Bowdler
22676,22677,Bromfield
22677,22678,Willaston on the Wirral


In [117]:
city_table = (
    clean_df[["city", "country"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

city_table["city_id"] = city_table.index + 1

country = (
    city_table[["city_id", "country"]]
    .copy()
    .reset_index(drop=True)
)

country["country_id"] = country.index + 1
country.rename(columns={"country": "country_name"}, inplace=True)

country = country[["country_id", "city_id", "country_name"]]
country

,country_id,city_id,country_name
0,1,1,France
1,2,2,France
2,3,3,France
3,4,4,France
4,5,5,France
...,...,...,...
22693,22694,22694,United Kingdom
22694,22695,22695,United Kingdom
22695,22696,22696,United Kingdom
22696,22697,22697,United Kingdom


In [31]:
country.to_csv("country.csv", index=False)

#### LOCATION TABLE

In [36]:
location = clean_df[["latitude", "longitude"]]
location.head()

,latitude,longitude
0,45.961674,1.169131
1,45.957040,1.205480
2,46.635895,1.386133
3,45.642610,1.824460
4,44.208860,2.960470


In [ ]:
location.to_csv("location.csv", index=False)

#### RESTAURANT LANGUAGE TABLE

In [37]:
restaurant_language = clean_df[["", ""]]
restaurant_language.head()

KeyError: "None of [Index(['', ''], dtype='object')] are in the [columns]"

In [ ]:
restaurant_language.to_csv("restaurant_language.csv", index=False)

#### RESTAURANT MEALS TABLE

In [ ]:
restaurant_meals = clean_df[["", ""]]
restaurant_meals.head()

In [ ]:
restaurant_meals.to_csv("restaurant_meals.csv", index=False)

#### RESTAURANT DIETARY OPTIONS TABLE

In [ ]:
restaurant_dietary_options = clean_df[["", ""]]
restaurant_dietary_options.head()

In [ ]:
restaurant_dietary_options.to_csv("restaurant_dietary_options.csv", index=False)

## Create CSV for the restaurant table

In [ ]:
"""
Restaurant: id, name, claimed, awards, open_days_per_week, original_open_hours, location_id, price_id, review_id
"""

In [33]:
restaurant = clean_df[["restaurant_name", "claimed", "awards", "open_days_per_week", "original_open_hours"]]
restaurant.tail()

,restaurant_name,claimed,awards,open_days_per_week,original_open_hours
326947,Domino's Pizza,Unclaimed,Unknown,7.0,"{""Mon"": [""11:00-23:00""], ""Tue"": [""11:00-23:00""..."
326948,Subway,Unclaimed,Unknown,7.0,"{""Mon"": [""10:00-16:00""], ""Tue"": [""10:00-16:00""..."
326949,The Kings Arms,Claimed,Unknown,7.0,"{""Mon"": [""16:00-22:00""], ""Tue"": [""16:00-22:00""..."
326950,Java Bar,Claimed,Unknown,5.0,"{""Mon"": [""07:00-14:00""], ""Tue"": [""07:00-14:00""..."
326951,Cobbles Kitchen and Deli,Claimed,Certificate of Excellence 2017,7.0,"{""Mon"": [""10:00-15:00""], ""Tue"": [""10:00-15:00""..."


In [ ]:
restaurant.to_csv("restaurant.csv", index=False)